In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#nltk.download()
import numpy as np
import tflearn
import tensorflow as tf
import random

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [8]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [9]:

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(32, 'documents')
(6, 'classes', [u'healthproblems_me', u'healthproblems_others', u'jobloss_me', u'jobloss_others', u'naturaldisasters_me', u'naturaldisasters_others'])
(60, 'unique stemmed words', [u"'s", u'.', u'accid', u'am', u'an', u'ar', u'attack', u'bad', u'broth', u'cant', u'company', u'country.he', u'country.lost', u'cousin', u'coutnry.feeling', u'coutnry.real', u'depress', u'earthquak', u'everyth', u'feel', u'fir', u'friend', u'from', u'got', u'had', u'he', u'heart', u'hur', u'i', u'in', u'is', u'job', u'last', u'lost', u'met', u'my', u'now', u'occ', u'our', u'pink', u'quit', u'rel', u'sad', u'scar', u'sev', u'sist', u'slip', u'stabl', u'stag', u'suff', u'tens', u'the', u'thi', u'tomorrow.feeling', u'unc', u'very', u'week', u'week.feeling', u'with', u'yesterday'])


In [11]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
#Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.40700 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.40700 - acc: 0.9465 -- iter: 24/32
Training Step: 4000  | total loss: 0.37726 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.37726 - acc: 0.9519 -- iter: 32/32
--
INFO:tensorflow:/home/ignitelab/Desktop/app/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [14]:
p = bow("i lost job", words)
print (p)
print (classes)

print(model.predict([p]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'healthproblems_me', u'healthproblems_others', u'jobloss_me', u'jobloss_others', u'naturaldisasters_me', u'naturaldisasters_others']
[[ 0.01997431  0.00534359  0.75159341  0.04773033  0.1432983   0.03206006]]


In [15]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )